In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('logistic-regression').getOrCreate()

In [3]:
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer

In [4]:
training = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0)
], ["id", "text", "label"])

In [5]:
tokenizer = Tokenizer(inputCol='text', outputCol='word')
hashingTF =HashingTF(inputCol=tokenizer.getOutputCol(), outputCol='features')

In [7]:
lr = LogisticRegression(maxIter=30, regParam=0.001)

In [8]:
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

In [9]:
model = pipeline.fit(training)

In [10]:
test = spark.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "spark hadoop spark"),
    (7, "apache hadoop")
], ["id", "text"])

In [11]:
prediction = model.transform(test)

In [12]:
prediction.select(['id', 'text', 'probability', 'prediction']).show()

+---+------------------+--------------------+----------+
| id|              text|         probability|prediction|
+---+------------------+--------------------+----------+
|  4|       spark i j k|[0.63102699631690...|       0.0|
|  5|             l m n|[0.98489377609773...|       0.0|
|  6|spark hadoop spark|[0.13563147748816...|       1.0|
|  7|     apache hadoop|[0.99563405823116...|       0.0|
+---+------------------+--------------------+----------+

